In [1]:
API_KEY="YOUR_API_KEY"

In [ ]:
import base64
import requests
from google.cloud import storage

# Extract image embedding
def getImageEmbeddingFromImageContent(content):
  base64EncodedContent = base64.b64encode(content)

  request="""
  {
    "requests": [
      {
        "image": {
          "content": "BASE64_ENCODED_DATA"
        },
        "features": [
          {
            "type": "IMAGE_EMBEDDING",
          }
        ]
      }
    ]
  }""".replace("BASE64_ENCODED_DATA", base64EncodedContent.decode('utf-8'))

  r = requests.post('https://us-vision.googleapis.com/v1/images:annotate?key=' + API_KEY, data=request)

  return r.json()["responses"][0]["imageEmbeddingVector"]["imageEmbeddingVector"]


def getImageEmbeddingFromGcsObject(gcsBucket, gcsObject):
  client = storage.Client()
  bucket = client.bucket(gcsBucket)
  blob = bucket.blob(gcsObject)

  with blob.open("rb") as f:
    return getImageEmbeddingFromImageContent(f.read())

def getImageEmbeddingFromFile(filePath):
  with open(filePath, "rb") as f:
    return getImageEmbeddingFromImageContent(f.read())

# Extract text embedding
def getTextEmbedding(text):
  request="""
  {
    "requests": [
      {
        "image": {
          "source": {
            "imageUri": "https://fileinfo.com/img/ss/xl/jpeg_43.png"
          }
        },
        "features": [
          {
            "type": "IMAGE_EMBEDDING",
          }
        ],
        "imageContext": {
              "imageEmbeddingParams": {
                  "contextualTexts": ["TEXT_TO_EXTRACT_EMBEDDING"]
              }
        }
      }
    ]
  }""".replace("TEXT_TO_EXTRACT_EMBEDDING", text)

  r = requests.post('https://us-vision.googleapis.com/v1/images:annotate?key=' + API_KEY, data=request)

  return r.json()["responses"][0]["imageEmbeddingVector"]["contextualTextEmbeddingVectors"][0]

: 

In [3]:
from google.cloud import storage
import csv

# This is the GCS bucket that holds the images that you want to analyze and
# index. You will need the bucket list and object reading permission to proceed.
# The default bucket provided here contains 61 images contributed by the
# engineer team. If you want to try your own image set, feel free to point this
# to another GCS bucket that holds your images. Please make sure all files in
# the GCS bucket are images (e.g. JPG, PNG). Non image files would cause
# inference exception down below.
IMAGE_SET_BUCKET_NAME = "coca-embedding-test-images" # @param {type: "string"}

gcsBucket = storage.Client().get_bucket(IMAGE_SET_BUCKET_NAME)

with open('image_embedding.csv', 'w') as f:
  csvWriter = csv.writer(f)
  csvWriter.writerow(['gcsUri', 'embedding'])
  for blob in gcsBucket.list_blobs():
    gcsUri = "gs://" + IMAGE_SET_BUCKET_NAME + "/" + blob.name
    print("Processing {}".format(gcsUri))
    embedding = getImageEmbeddingFromGcsObject(IMAGE_SET_BUCKET_NAME, blob.name)
    csvWriter.writerow([gcsUri, str(embedding)])

OSError: Project was not passed and could not be determined from the environment.

In [ ]:
from google.colab import files
import os

if not os.path.exists('image_embedding.csv'):
  print("Upload your saved image_embedding.csv")

  uploaded = files.upload()

  found_file = False
  for filename in uploaded.keys():
    print(uploaded)
    if filename == "image_embedding.csv":
      print("Found your image_embedding.csv")
      found_file = True

  if not found_file:
    raise ValueError("No image_embedding.csv uploaded")

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('image_embedding.csv')
df.embedding = df.embedding.apply(eval).apply(np.array)

df.head(5)

## Define a Helper function to search image by text

In [ ]:
from pandas.io.parsers.readers import ParserBase
import time
import re
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files

# @@search_backend_function is a function that takes two parameters
#   @@embedding_vector: a embedding vector to search against.
#   @@num_neighbors: number of neighbors to return from the backend.
# and it returns two things
#   @@neighbors: a list of ids (0 based position in the dataset) indicating the
#                neighbors that's closest to the input embedding_vector.
#   @@distances: a list of distances, each corresponding to the distance from
#                the @@embedding_vector to the data point int he dataset,
#                indexed by the corresponding id in @@neighbor.
def searchImagesByEmbedding(start_time, embedding, search_backend_function,
                            num_neighbors = 3):
    neighbors, distances = search_backend_function(
        embedding, num_neighbors)
    end = time.time()

    gcsClient = storage.Client()
    for id, dist in zip(neighbors, distances):
        print(f'docid:{id} dist:{dist} gcsUri:{df.gcsUri[id]}')
        # Display the image
        gcsUri = df.gcsUri[id]
        m = re.search('gs://([^/]*)/([^$]*)', gcsUri)
        imageBlob = gcsClient.get_bucket(m[1]).blob(m[2])
        tmpFilename = "/tmp/tmp_image"
        imageBlob.download_to_filename(tmpFilename)
        image = cv2.imread(tmpFilename, -1)
        cv2_imshow(image)

    print("Latency (ms):", 1000*(end - start_time))

def searchImagesByText(query, search_backend_function, num_neighbors = 3):
    start_time = time.time()
    query_embedding = getTextEmbedding(query)
    return searchImagesByEmbedding(start_time, query_embedding,
                                   search_backend_function)

def searchImagesByUploadedImages(search_backend_function, num_neighbors = 3):
    uploaded = files.upload()
    for filename in uploaded.keys():
      print('Searching images similar to {}'.format(filename))
      image = cv2.imread(filename, -1)
      cv2_imshow(image)
      start_time = time.time()
      image_embedding = getImageEmbeddingFromFile(filename)
      searchImagesByEmbedding(start_time, image_embedding,
                              search_backend_function, num_neighbors)


## Construct DAtaset that ScaNN consumes as the input.

In [ ]:
import scann

# df.shape[0] is the #data in the dataset.
# df.embedding[0].size is the embedding vector size.
dataset = np.empty((df.shape[0], df.embedding[0].size))
for i in range(df.shape[0]):
  dataset[i] = df.embedding[i]

searcher = scann.scann_ops_pybind.builder(dataset, 10, "dot_product").tree(
    num_leaves=10, num_leaves_to_search=10).score_ah(2).reorder(100).build()

def searchByScaNN(embedding_vector, num_neighbors):
    return searcher.search(
        embedding_vector, final_num_neighbors = num_neighbors)

In [ ]:
# Search by text. Modify the query and execute to see the search result.
searchImagesByText("lake view", searchByScaNN)

In [ ]:
# Search by image. Upload your own image and search.
searchImagesByUploadedImages(searchByScaNN)

## Search Using Matching Engine

In [ ]:
REGION = 'us-central1' # @param {type: "string"}
# The embedding file, index file and VME resources are going to be created in
# this project.
PROJECT_ID = 'cloud-llm-preview1' # @param {type: "string"}

! gcloud config set project {PROJECT_ID}

ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

# Bucket for holding the indexing data for VME. You will need the bucket
# writing permission to proceed.
INDEX_DATA_BUCKET_NAME = '' # @param {type: "string"}

! gsutil mb -l $REGION -p $PROJECT_ID gs://{INDEX_DATA_BUCKET_NAME}
! gsutil rm -raf gs://{INDEX_DATA_BUCKET_NAME}/** 2> /dev/null || true

### Convert the Image embedding into the JSON

In [ ]:
import json

with open('image_embedding.json', 'w') as f:
  for idx, embedding in enumerate(df.embedding):
    json.dump({"id": idx, "embedding": embedding.tolist()}, f)
    f.write('\n')

! gsutil cp image_embedding.json gs://{INDEX_DATA_BUCKET_NAME}/image_embedding.json

## Create VME Index

In [ ]:
import time
from google.cloud import aiplatform_v1beta1
from google.protobuf import struct_pb2

DIMENSIONS = df.embedding[0].size
DISPLAY_NAME = "image-embedding"

index_client = aiplatform_v1beta1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=df.shape[0]),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=7),
    }
)

algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=10),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value="gs://{}".format(INDEX_DATA_BUCKET_NAME)),
    }
)

matching_engine_index = {
    "display_name": DISPLAY_NAME,
    "description": "Google Products Vertex AI Matching Engine Index",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

# Create the index and wait for it to be ready.
matching_engine_index_operation = index_client.create_index(
    parent=PARENT, index=matching_engine_index
)

while not matching_engine_index_operation.done():
    print("Poll the operation to create index...")
    time.sleep(60)

INDEX_RESOURCE_NAME = matching_engine_index_operation.result().name
print("Index created: {}".format(INDEX_RESOURCE_NAME))

### Create Index Endpoint and Deploy

In [ ]:
index_endpoint_client = aiplatform_v1beta1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

index_endpoint = {
    "display_name": "coca_image_index_endpoint",
    "public_endpoint_enabled": True,
}

index_endpoint_operation = index_endpoint_client.create_index_endpoint(
    parent=PARENT, index_endpoint=index_endpoint
)

while not index_endpoint_operation.done():
    print("Poll the operation to create index endpoint...")
    time.sleep(60)

INDEX_ENDPOINT_NAME = index_endpoint_operation.result().name
print("Index endpoint created: {}".format(INDEX_ENDPOINT_NAME))

index_endpoint = index_endpoint_client.get_index_endpoint(
    name = INDEX_ENDPOINT_NAME)

INDEX_ENDPOINT_PUBLIC_DOMAIN_NAME = index_endpoint.public_endpoint_domain_name

print("Index endpoint public domain name: {}".format(
    INDEX_ENDPOINT_PUBLIC_DOMAIN_NAME))

In [ ]:
DEPLOYED_INDEX_ID = "coca_image_embedding_deployment" # @param {type: "string"}

deploy_matching_engine_index = {
    "id": DEPLOYED_INDEX_ID,
    "display_name": DEPLOYED_INDEX_ID,
    "index": INDEX_RESOURCE_NAME,
}

deploy_index_operation = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_matching_engine_index
)

while not deploy_index_operation.done():
    print("Poll the operation to deploy index...")
    time.sleep(60)

deploy_index_operation.result()

In [ ]:
from proto.fields import RepeatedField, ProtoType
from google.cloud.aiplatform_v1beta1 import MatchServiceClient, IndexDatapoint, FindNeighborsRequest, FindNeighborsResponse

match_service_client = MatchServiceClient(
    client_options=dict(api_endpoint=INDEX_ENDPOINT_PUBLIC_DOMAIN_NAME)
)

def searchByVertexMatchingEngine(embedding_to_search, neighbor_count):
  datapoint = IndexDatapoint(
      feature_vector = embedding_to_search
  )

  query = FindNeighborsRequest.Query(
      datapoint = datapoint,
      neighbor_count = neighbor_count
  )

  find_neighbors_request = FindNeighborsRequest(
      index_endpoint = INDEX_ENDPOINT_NAME,
      deployed_index_id = DEPLOYED_INDEX_ID,
      queries = [query],
  )

  response = match_service_client.find_neighbors(
      request = find_neighbors_request)

  neighbors = []
  distances = []
  for neighbor in response.nearest_neighbors[0].neighbors:
    neighbors.append(int(neighbor.datapoint.datapoint_id))
    distances.append(neighbor.distance)

  return neighbors, distances

### Search

In [ ]:
# Search by text. Modify the query and execute to see the search result.
searchImagesByText("lake view", searchByVertexMatchingEngine)

In [ ]:
# Search by image. Upload your own image and search.
searchImagesByUploadedImages(searchByVertexMatchingEngine)